In [1]:
import json

train_data_path = "data/train.json"

train_data = []
with open(train_data_path, 'r', encoding='utf-8') as f:
    for line in f:
        if line.strip():  # 忽略空行
            train_data.append(json.loads(line.strip()))

print(f"共加载训练样本数：{len(train_data)}")
print("样本:\ncontext:",train_data[0]['context'],"\nquestion:",train_data[0]['question'],"\nanswer:",train_data[0]['answer'])

共加载训练样本数：14520
样本:
context: 第35集雪见缓缓张开眼睛，景天又惊又喜之际，长卿和紫萱的仙船驶至，见众人无恙，也十分高兴。众人登船，用尽合力把自身的真气和水分输给她。雪见终于醒过来了，但却一脸木然，全无反应。众人向常胤求助，却发现人世界竟没有雪见的身世纪录。长卿询问清微的身世，清微语带双关说一切上了天界便有答案。长卿驾驶仙船，众人决定立马动身，往天界而去。众人来到一荒山，长卿指出，魔界和天界相连。由魔界进入通过神魔之井，便可登天。众人至魔界入口，仿若一黑色的蝙蝠洞，但始终无法进入。后来花楹发现只要有翅膀便能飞入。于是景天等人打下许多乌鸦，模仿重楼的翅膀，制作数对翅膀状巨物。刚佩戴在身，便被吸入洞口。众人摔落在地，抬头发现魔界守卫。景天和众魔套交情，自称和魔尊重楼相熟，众魔不理，打了起来。 
question: 仙剑奇侠传3第几集上天界 
answer: 第35集


In [2]:
from datasets import Dataset

# 构造 input_text 和 target_text
samples = []
for item in train_data:
    input_text = f"问题：{item['question']} 文本：{item['context']}"
    target_text = item['answer']
    samples.append({
        "input_text": input_text,
        "target_text": target_text
    })

# 转换为 HuggingFace Dataset 并划分训练/验证集
dataset = Dataset.from_list(samples)
dataset = dataset.train_test_split(test_size=0.1, seed=42)

train_dataset = dataset["train"]
eval_dataset = dataset["test"]

print("训练集样本数：", len(train_dataset))
print("验证集样本数：", len(eval_dataset))

/root/miniconda3/envs/t5_base_qa_env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


训练集样本数： 13068
验证集样本数： 1452


In [14]:
from transformers import T5Tokenizer, T5ForConditionalGeneration

# model_name = "Langboat/mengzi-t5-base"
model_path = "/root/.cache/modelscope/hub/models/langboat/mengzi-t5-base"
tokenizer = T5Tokenizer.from_pretrained(model_path)
model = T5ForConditionalGeneration.from_pretrained(model_path)

max_input_len = 512
max_target_len = 64

def preprocess_function(examples):
    model_inputs = tokenizer(
        examples["input_text"],
        max_length=max_input_len,
        padding="max_length",
        truncation=True
    )

    labels = tokenizer(
        examples["target_text"],
        max_length=max_target_len,
        padding="max_length",
        truncation=True
    )["input_ids"]

    # 避免对 PAD 部分计算损失
    labels = [
        [(l if l != tokenizer.pad_token_id else -100) for l in label]
        for label in labels
    ]

    model_inputs["labels"] = labels
    return model_inputs

In [4]:
tokenized_train = train_dataset.map(preprocess_function, batched=True)
tokenized_eval = eval_dataset.map(preprocess_function, batched=True)

tokenized_train.set_format("torch")
tokenized_eval.set_format("torch")

Map:   0%|          | 0/13068 [00:00<?, ? examples/s]

Map: 100%|██████████| 1452/1452 [00:00<00:00, 1547.09 examples/s]


In [5]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="./mengzi_t5_qa_output",
    run_name="mengzi-t5-qa-run",
    num_train_epochs=6,                        
    per_device_train_batch_size = 4,
    per_device_eval_batch_size = 4, 
    eval_accumulation_steps=4,                 
    gradient_accumulation_steps=1,
    eval_strategy="epoch",                     
    save_strategy="epoch",
    learning_rate=3e-5,
    warmup_steps=500,
    weight_decay=0.01,
    save_total_limit=2,
    load_best_model_at_end=True,    
    metric_for_best_model="eval_loss",      
    greater_is_better=False,                  
    logging_dir="./logs",
    logging_steps=50,
    report_to="none",
    fp16=True,
    dataloader_num_workers=4,
    seed=42
)
from transformers import Trainer, EarlyStoppingCallback

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_eval,
    tokenizer=tokenizer,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=2)]
)

In [6]:
trainer.train()
# 保存模型和分词器
output_dir = "./mengzi_t5_qa_model"
trainer.save_model(output_dir)
tokenizer.save_pretrained(output_dir)

Epoch,Training Loss,Validation Loss
1,0.586000,0.582998
2,0.360000,0.608305
3,0.200000,0.707420


There were missing keys in the checkpoint model loaded: ['encoder.embed_tokens.weight', 'decoder.embed_tokens.weight'].


('./mengzi_t5_qa_model/tokenizer_config.json',
 './mengzi_t5_qa_model/special_tokens_map.json',
 './mengzi_t5_qa_model/spiece.model',
 './mengzi_t5_qa_model/added_tokens.json')